# I want to actually make an LSTM
I will use the data in entero_unique_arg.csv, break each entry up into a list of 14 long characters that move over a space each time. The 15th character will be the one that the neural net is trying to predict. 


For each line of the input data I want to give it a 12 position long random "seed" along with a "start" character which I have decided is ":".

In [55]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Tensor Flow Version: 2.4.0
Keras Version: 2.4.0

Python 3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) 
[GCC 9.4.0]
Pandas 1.3.3
Scikit-Learn 1.0
GPU is NOT AVAILABLE


2021-10-15 08:43:18.846065: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [56]:
import pandas as pd
import numpy as np

np.random.seed(42)
pd.set_option('display.max_rows', 1000)
pd.set_option( 'display.max_colwidth', None )

df = pd.read_csv('data/entero_unique_arg.csv', skiprows=3, names=['sequences'])
FRAG_SIZE = 14

display(df.head())

,sequences
0,GCATCCGTAGTTCAGTTGGAtAGAGCACTCGGCTACGAACCGAGAGGtCGGAGGTTCAAATCCTTCCGGATGCA
1,GCGCTTGTAGCTCAGTTGGAtAGAGCGCTACCCTCCGAAGGTAGAGGcCTCAGGTTCGAATCCTGTCAAGCGCA
2,GCGCTCTTAGCTCAGATGGAtAGAGCAACGGCCTTCTAAGCCGTAGGtCATAGGTTCGAATCCTATAGAGCGCA
3,GCGCCCTTAGCTCAGTTGGAtAGAGCAACGACCTTCTAAGTCGTGGGcCGCAGGTTCGAATCCTGCAGGGCGCG
4,GTCCTCTTAGTTAAATGGAtATAACGAGCCCCTCCTAAGGGCTAAtTGCAGGTTCGATTCCTGCAGGGGACA


In [57]:
def add_prefix(sequence):
    prefix = np.random.rand(1,FRAG_SIZE-1).tolist()[0]
    prefix.append(':')
    prefix.extend(sequence)
    return prefix

def add_suffix(sequence):
    sequence.append(';')
    return sequence

def split_sequence(sequence):
    return [str(x) for x in sequence]

def make_X(sequence):
    steps = len(sequence)-FRAG_SIZE
    train = []
    for step in range(steps):
        train.append( sequence[step:FRAG_SIZE+step+1] )
    return train
    
def make_y(sequence):
    steps = len(sequence)-FRAG_SIZE+1
    predict = []
    for step in range(steps):
        predict.append( sequence[FRAG_SIZE+step-1] )
    return predict

def make_dummy(item):
    try:
        return uniqueChars[item]
    except KeyError:
        return item
            

In [58]:
example = [0.3745401188473625,
  0.9507143064099162,
  0.7319939418114051,
  0.5986584841970366,
  0.15601864044243652,
  0.15599452033620265,
  0.05808361216819946,
  0.8661761457749352,
  0.6011150117432088,
  0.7080725777960455,
  0.020584494295802447,
  0.9699098521619943,
  0.8324426408004217,
  ':',
  'G',
  'C',
  'A',
  'T',
  'C',
  'C',
  'G',
  'T',
  'A',
  'G',
  'T',
  'T',
  'C',
  'A',
  'G',
  'T',
  'T',
  'G',
  'G',
  'A',
  't',
  'A',
  'G',
  'A',
  'G',
  'C',
  'A',
  'C',
  'T',
  'C',
  'G',
  'G',
  'C',
  'T',
  'A',
  'C',
  'G',
  'A',
  'A',
  'C',
  'C',
  'G',
  'A',
  'G',
  'A',
  'G',
  'G',
  't',
  'C',
  'G',
  'G',
  'A',
  'G',
  'G',
  'T',
  'T',
  'C',
  'A',
  'A',
  'A',
  'T',
  'C',
  'C',
  'T',
  'T',
  'C',
  'C',
  'G',
  'G',
  'A',
  'T',
  'G',
  'C',
  'A',
  ';']

In [59]:
df['sequences'] = df['sequences'].map( split_sequence )
df['sequences'] = df['sequences'].map( add_prefix )
df['sequences'] = df['sequences'].map( add_suffix )
#display( df['sequences'].map( add_prefix ) )

In [60]:
X = []
y = []
for sequence in df['sequences']:
    X.extend(make_X(sequence))

In [61]:
columns = [f'X_{x}' for x in range(FRAG_SIZE)]
columns.append('y')

dataset = pd.DataFrame(X, columns=columns)
display(dataset.head())
dataset.shape

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.37454,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C,A
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C,A,T
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.96991,0.832443,:,G,C,A,T,C


(16519, 15)

In [62]:
# Just looking how many tRNAs start with each letter
dataset[dataset['X_13']==':'].groupby('y')['y'].count()

y
A      3
C     11
G    202
T      6
Name: y, dtype: int64

In [63]:
# Lets get the basic stats that Jeff was getting
chars = df['sequences'].map(lambda calc: len(calc)).sum()
uniqueChars = {':': 0, 'A': 1, 'T': 2, 'G':3, 'C':4, 'a':5, 't':6, 'g':7, 'c':8, ';':9}
uniqueNums = {value:key for key, value in uniqueChars.items()}
display(uniqueNums)
uniqueChars

{0: ':',
 1: 'A',
 2: 'T',
 3: 'G',
 4: 'C',
 5: 'a',
 6: 't',
 7: 'g',
 8: 'c',
 9: ';'}

{':': 0,
 'A': 1,
 'T': 2,
 'G': 3,
 'C': 4,
 'a': 5,
 't': 6,
 'g': 7,
 'c': 8,
 ';': 9}

In [64]:
dataset = dataset.applymap(make_dummy)

In [65]:
#dataset.to_csv('processed_LSTM_dataset_chars.csv', index=False)
dataset

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0,3
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3,4
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4,1
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1,2
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1.000000,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16514,4.000000,1.000000,1.000000,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3,4
16515,1.000000,1.000000,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3.000000,4,3
16516,1.000000,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3.000000,4.000000,3,4
16517,3.000000,2.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.000000,3.000000,1.000000,3.000000,4.000000,3.000000,4,3


# I HAVE MADE MY DATASET
Now lets train the LSTM finally!!!

In [66]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
#from tensorflow.python.ops.math_ops import reduce_prod
import numpy as np
import pandas as pd
import random
import sys
import io
import requests
import re

In [67]:
df = pd.read_csv('data/processed_LSTM_dataset.csv')
display(df.head())
df.shape

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0,3
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3,4
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4,1
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1,2
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1.000000,2,4


(16519, 15)

In [68]:
df = df.astype({'X_13':'float64'})
df.dtypes

X_0     float64
X_1     float64
X_2     float64
X_3     float64
X_4     float64
X_5     float64
X_6     float64
X_7     float64
X_8     float64
X_9     float64
X_10    float64
X_11    float64
X_12    float64
X_13    float64
y         int64
dtype: object

In [69]:
df.head()

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
0,0.374540,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.0,3
1,0.950714,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.0,4
2,0.731994,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.0,1
3,0.598658,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1.0,2
4,0.156019,0.155995,0.058084,0.866176,0.601115,0.708073,0.020584,0.969910,0.832443,0.000000,3.000000,4.000000,1.000000,2.0,4


In [70]:
#Lets vectorize this (or not)
#We can also just leave it as our numbers

In [71]:
inputLength = len(  list(df.values.tolist())[1]  ) - 1
outputLength = len(uniqueChars)
rowLength = len(  list(df.values.tolist())  )
print('Input dim: {}\nOutput dim: {}\nTraining: {}'.format(inputLength, outputLength, rowLength))
#dataList = list(df.values.tolist())

Input dim: 14
Output dim: 10
Training: 16519


In [72]:
#print('Vectorization')
#x = np.zeros(( rowLength, inputLength, outputLength ), dtype=object)
#y = np.zeros(( rowLength, outputLength ), dtype=object)

#for i, sentence in enumerate(dataList):
#    sentence = sentence[:-1]
#    nextChar = sentence[-1]
#    for t, char in enumerate(sentence):
#        x[i,t,uniqueChars[char]] = 1
#    y[i,uniqueChars[nextChar]]

In [73]:
mask = df.columns.str.contains('X')
X = np.array( list(df.loc[:,mask].values.tolist()), dtype='float64' )
y = np.array( pd.get_dummies( list(df['y'].values.tolist()) ) )

In [74]:
# I think this is why y=/=10, it is because ':' never appears in the y column, so outputdim is actually 9!
display(X.shape)
display(y[0])
print(df['y'].values.tolist()[0], uniqueChars['G'])
df.groupby('y')['y'].count().to_dict()
{uniqueNums[key]:value for key, value in df.groupby('y')['y'].count().to_dict().items()}

(16519, 14)

array([0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=uint8)

3 3


{'A': 3554,
 'T': 3251,
 'G': 4768,
 'C': 4278,
 'a': 10,
 't': 416,
 'g': 3,
 'c': 17,
 ';': 222}

The above stuff shows the spread of each of the different characters in the neural nets vocabulary. Strange that it never says C considering that is its second highest vocab
===

The way I understand this next part is that
* samples: well how many samples there are
* timesteps: Having vectorized the data, how many points before the one we trying to predict
* features: This one is a bit tough, but I think it has to do with dummy variables

In [75]:
#                 samples,   timesteps,  features   ::: https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Here is we actually frame the probelm
I followed Jeff Heaton, and he had a categorical problem. But...I think if I change how my y is, I can frame this as a regression! So I would have one output neuron and I would use a different loss function, like maybe RMS or sum of square roots whatever (gonna have to rewatch his vid)

In [80]:
print('Build model')
model = Sequential()
model.add(LSTM(128, input_shape=(14, 1)))
model.add( Dense(128, activation="relu" ) )
model.add( Dense(9, activation='softmax') )

optimizer = RMSprop(lr=0.05)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model


In [81]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 1161      
Total params: 84,233
Trainable params: 84,233
Non-trainable params: 0
_________________________________________________________________


In [82]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [83]:
# Left off at 16:31 of the lstm generate text video...
print(uniqueChars)
#del uniqueChars[':']
uniqueNums = {value-1:key for key,value in uniqueChars.items()}
print(uniqueChars)
print(uniqueNums)
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    #start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [1.0, 1.2, 1.5, 1.8, 2]:
        print('----- temperature:', temperature)

        generated = ''
        
        sentence = np.random.rand(1,FRAG_SIZE-1).tolist()[0]
        sentence.append(0)
        #print(sentence)
        #sentence = np.array(sentence, dtype='float64')
        #print(sentence)
        
        generated = ''
        print(f'----- Generating with seed: "{sentence}"')
        #sys.stdout.write(sentence)

        for i in range(80):
            x_pred = np.zeros((1,FRAG_SIZE,1))
            for t, char in enumerate(sentence):
                #try:
                 #   x_pred[0,t,0] = uniqueNums[char] ################################
                #except KeyError:
                x_pred[0,t,0] = char

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = uniqueNums[next_index]

            generated += next_char
            sentence = sentence[1:] + next_index

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

{'A': 1, 'T': 2, 'G': 3, 'C': 4, 'a': 5, 't': 6, 'g': 7, 'c': 8, ';': 9}
{'A': 1, 'T': 2, 'G': 3, 'C': 4, 'a': 5, 't': 6, 'g': 7, 'c': 8, ';': 9}
{0: 'A', 1: 'T', 2: 'G', 3: 'C', 4: 'a', 5: 't', 6: 'g', 7: 'c', 8: ';'}


In [ ]:
# Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# See https://github.com/tensorflow/tensorflow/issues/31308
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(X, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
130/130 [==============================] - 3s 17ms/step - loss: 4.2257
******************************************************
----- Generating text after Epoch: 0
----- temperature: 1.0
----- Generating with seed: "[0.5069204379618346, 0.40941207181701833, 0.8108785810079184, 0.835825927910516, 0.3321910331156046, 0.6936180574737952, 0.7711228158518866, 0.6546506868659545, 0.15159507480009937, 0.875883083517057, 0.5390945527360087, 0.2824724263700037, 0.4252281854376939, 0]"
GGTGCGGGCAGGGTGGGTGTGGCGAGGGTGCGGGGGGTGGGGGGTGGGTGAGGGGTGAGGTGGCTGGGCGAGCGGGGGGG
----- temperature: 1.2
----- Generating with seed: "[0.6067317247174364, 0.6728716685980798, 0.08114917853623915, 0.9514907101767872, 0.8384918338430248, 0.8050903184956627, 0.8229838050417675, 0.9327105081327614, 0.5442539661136977, 0.2002820178676259, 0.6167836078525412, 0.742881493667939, 0.7379173933540254, 0]"
GGGGCGGGGAGGGTGGGGTAGAGGGGGGGGGTGGGCGGGGGGCGGGGGGGTGTCTGGAGGGGGGGGGGCGTGGGCGGGTG
----- temperature: 1.5
----- G

/tmp/ipykernel_624/64110448.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


AGGTGGGTGGGGAGGGGGGGGGGCGGGAGAGGAGGCAGGGGGGGGGGGGGGGGGGGGGCGGGGGGGTGGGGG
----- temperature: 1.2
----- Generating with seed: "[0.4940428669357333, 0.055217626750405646, 0.9546375133600558, 0.12857726079687848, 0.6974612672267269, 0.5062310360908802, 0.025632249884422587, 0.9247474758643617, 0.8518368088189332, 0.5242579279038355, 0.22894233613944526, 0.12162289068009424, 0.3633924794321266, 0]"
CGTtAGGGCGAGGCGAGGGGGGGGGGGGGGGGGGCGGGCGGGGGGCGAGCGGGGGGGAGGACATCGGGCGCGGGTCGGGG
----- temperature: 1.5
----- Generating with seed: "[0.9115968865602742, 0.3682627898719132, 0.0455501559653525, 0.30022711154138637, 0.5311094225441555, 0.40822182258823203, 0.27384251909925916, 0.13132324976431742, 0.8191645000962298, 0.9961452421241919, 0.32954213352388206, 0.3816210441766039, 0.618037524676957, 0]"
CGTTGTTGGGGCGGCGGGCAGGGGCGGGGCACCCGGGGAGTAGGCGGTGAGGGGGGGGTTCGCTGGGGTGAGGGGGGTGG
----- temperature: 1.8
----- Generating with seed: "[0.9515803094398696, 0.9448354530207373, 0.7796086425424651, 0.60210

In [47]:
X.shape

(16519, 14, 1)

In [48]:
len(uniqueChars)

10

In [49]:
uniqueChars

{':': 0,
 'A': 1,
 'T': 2,
 'G': 3,
 'C': 4,
 'a': 5,
 't': 6,
 'g': 7,
 'c': 8,
 ';': 9}

In [330]:
uniqueNums

{0: ':',
 1: 'A',
 2: 'T',
 3: 'G',
 4: 'C',
 5: 'a',
 6: 't',
 7: 'g',
 8: 'c',
 9: ';'}